# Register model in Azure Machine learning

In [22]:
from azureml.core import Workspace

from lib import get_client,get_workspace
client=get_client()
print(f'workspace name: {client.workspace_name}')
datastore=client.datastores.get_default()
print(f'datastore name: {datastore.name} type: {datastore.type}')

workspace name: azure-ml
datastore name: storecapture type: DatastoreType.AZURE_DATA_LAKE_GEN2


In [23]:
ws=get_workspace()
experiments=ws.experiments.get('grstb-signal-classification')

{'_workspace': Workspace.create(name='azure-ml', subscription_id='7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d', resource_group='azure-ml'),
 '_name': 'grstb-signal-classification',
 '_workspace_client': <azureml._restclient.workspace_client.WorkspaceClient at 0x151c12150>,
 '_id': '8fcd5cfd-dde2-4987-a6cb-5804251a160c',
 '_archived_time': None,
 '_experiment_dto': <azureml._restclient.models.experiment_dto.ExperimentDto at 0x132d42690>,
 '_tags': {},
 '_identity': 'Experiment#grstb-signal-classification',
 '_logger': <TraceLogger azureml.Experiment#grstb-signal-classification (WARNING)>,
 '_portal_url': 'https://ml.azure.com',
 '_formatted_tid': '&tid=5d250ee6-9438-4941-b7c3-dab39198c26a',
 '_workspace_url': 'https://ml.azure.com?wsid=/subscriptions/7db1ee9f-abea-4b8b-b5a9-e6e82ecfa19d/resourcegroups/azure-ml/workspaces/azure-ml&tid=5d250ee6-9438-4941-b7c3-dab39198c26a',
 '_experiment_url': 'https://ml.azure.com/experiments/id/8fcd5cfd-dde2-4987-a6cb-5804251a160c?wsid=/subscriptions/7db1ee9f-

In [24]:
runs=experiments.get_runs()
for run in runs:
    print(run)

Run(Experiment: grstb-signal-classification,
Id: gray_door_slb5k9v3yl,
Type: automl,
Status: Completed)


In [45]:
import mlflow

MLFLOW_TRACKING_URI=ws.get_mlflow_tracking_uri()
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

from mlflow.tracking.client import MlflowClient

mlflow_client = MlflowClient()
mlflow_parent_run = mlflow_client.get_run('gray_door_slb5k9v3yl')

# Fetch the id of the best automl child trial.
target_checkpoint_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print(target_checkpoint_run_id)

gray_door_slb5k9v3yl_HD_0


In [46]:
best_child_run_id = mlflow_parent_run.data.tags["automl_best_child_run_id"]
print(f"Found best child run id: {best_child_run_id}")

best_run = mlflow_client.get_run(best_child_run_id)

print("Best child run: ")
print(best_run)

Found best child run id: gray_door_slb5k9v3yl_HD_0
Best child run: 
<Run: data=<RunData: metrics={'AUC_macro': 1.0,
 'AUC_micro': 1.0,
 'AUC_weighted': 1.0,
 'accuracy': 0.99962,
 'accuracy_train': 0.9648,
 'average_precision_score_macro': 1.0,
 'average_precision_score_micro': 1.0,
 'average_precision_score_weighted': 1.0,
 'f1_score_macro': 0.99955,
 'f1_score_micro': 0.99962,
 'f1_score_weighted': 0.99962,
 'log_loss': 0.00171,
 'log_loss_train': 0.10817,
 'precision_score_macro': 0.99941,
 'precision_score_micro': 0.99962,
 'precision_score_weighted': 0.99962,
 'recall_score_macro': 0.9997,
 'recall_score_micro': 0.99962,
 'recall_score_weighted': 0.99962}, params={}, tags={'hyperparameters': '{"ignored_argument": 0}',
 'mlflow.parentRunId': 'gray_door_slb5k9v3yl_HD',
 'mlflow.rootRunId': 'gray_door_slb5k9v3yl',
 'mlflow.runName': 'red_box_nt2pwhq8',
 'mlflow.source.name': 'hd_image_classification_dnn_driver.py',
 'mlflow.source.type': 'JOB',
 'mlflow.user': 'Jean-Louis Tournay'}>,

In [ ]:
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml.entities import Model
import datetime
model_name = "grstb-signal-classification"
online_endpoint_name = "grstb" + datetime.datetime.now().strftime(
    "%m%d%H%M%f"
)
model = Model(
    path=f"azureml://jobs/{best_run.info.run_id}/outputs/artifacts/outputs/mlflow-model/",
    name=model_name,
    description="Grstb signal classification model",
    type=AssetTypes.MLFLOW_MODEL,
)

registered_model = client.models.create_or_update(model)
print(registered_model)